<a href="https://colab.research.google.com/github/gabrielborja/parc_de_montjuic/blob/main/interactive_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Orders Visualization

In [ ]:
# Importing python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.express as px
from ipywidgets import interact
import matplotlib.ticker

## Loading Data

In [ ]:
# Loading data from local drive
from google.colab import files
uploaded1 = files.upload()

In [ ]:
# Storing loaded data from excel to a pandas dataframe
import io
df1 = pd.read_excel(io.BytesIO(uploaded1['Script_202209080812.xlsx']))

In [ ]:
# Chainging columns to lower case
df1.rename(columns={i:i.lower() for i in df1.columns}, inplace=True)

In [ ]:
# Creating time categories
df1 = df1.assign(year = df1['order_date'].dt.year,
                 quarter = df1['order_date'].dt.quarter,
                 month = df1['order_date'].dt.month,
                 month_name = df1['order_date'].dt.month_name(),
                 weeknum_order = df1['order_date'].dt.isocalendar().week,
                 weeknum_activation = df1['activation_date'].dt.isocalendar().week,
                 weeknum_cancellation = df1['cancellation_date'].dt.isocalendar().week,
                 day = df1['order_date'].dt.day_name()
                 )

In [ ]:
# Assign only smph
df1 = df1[(df1['product_family']=='Smartphone')&(df1['order_type']=='New RGU')].reset_index(drop=True)

In [ ]:
df1.order_type.unique()

In [ ]:
# Checking the dataframe info
df1.info()

## Visualization

In [ ]:
# Exploring list of columns with value counts
@interact(Column_name=df1.columns, Percentage=[True, False])
def explore_value_counts(Column_name, Percentage):
  df = df1.value_counts(subset=[Column_name], normalize=Percentage, dropna=False)
  return df.head()

In [ ]:
# Creating categories for interactive visualization
cat_list_1 = df1.loc[:,'product_family'].unique().tolist()
time_list_1 = ['order_date','activation_date','cancellation_date']
time_list_2 = ['weeknum_order','weeknum_activation','weeknum_cancellation']

In [ ]:
# Creating function to aggregate order dates
def aggregate_order_by_dates():
  df_list = []
  for i in cat_list_1:
    for j in time_list_1:
      df = df1[df1['product_family']==i].copy().reset_index(drop=True)
      df = df.groupby(by=[j]).agg(total=('order_no','count')).reset_index()
      df = df.rename(columns={j:'period'})
      df = df.assign(category=i, order_type=j[:-5], cum_sum=df['total'].cumsum())
      df_list.append(df)
  df_con = pd.concat(df_list, axis=0) #=> Stacking df on top of each others
  return df_con

df1_or = aggregate_order_by_dates()

In [ ]:
# Creating function to aggregate order dates by weeknum
def aggregate_by_weeknum():
  df_list = []
  for i in cat_list_1:
    for j in time_list_2:
      df = df1[df1['product_family']==i].copy().reset_index(drop=True)
      df = df.groupby(by=[j]).agg(total=('order_no','count')).reset_index()
      df = df.rename(columns={j:'weeknum'})
      df = df.assign(category=i, order_type=j[8:], cum_sum=df['total'].cumsum())
      df_list.append(df)
  df_con = pd.concat(df_list, axis=0) #=> Stacking df on top of each others
  return df_con

df1_or = aggregate_by_weeknum()

In [ ]:
df1_or[df1_or['category']=='Machine to machine'].groupby(['order_type']).agg(total = ('cum_sum','max'))

In [ ]:
# Exploring feature importance in same graph
@interact(Category=cat_list_1)
def plot_weekly_orders(Category):
  ''' Function for comparing product family orders agains time '''
  df = df1_or[df1_or['category']==Category].copy().reset_index(drop=True)
  fig_x, axes = plt.subplots(1, 1, figsize=(9.5, 6.5))
  axes.set_title(f'{Category}: 50K Hunt')
  sns.lineplot(data=df, x='weeknum', y='cum_sum', hue='order_type')#,ax=axes)
  for container in axes.containers:
    axes.bar_label(container, fontsize=12)
  #fig_x.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  #plt.xticks(rotation=30)
  plt.xticks(np.arange(15, 38, 1.0))
  plt.axvline(x=20, color='k', linestyle=':', linewidth=0.5)
  plt.axvline(x=26, color='k', linestyle=':', linewidth=0.5)
  plt.axvline(x=30, color='k', linestyle=':', linewidth=0.5)
  plt.axvline(x=35, color='k', linestyle=':', linewidth=0.5)
  plt.show()

In [ ]:
# Exploring sales channels
@interact(Sales_channel = ['sales_channel_name','sales_channel_chain_name','sales_channel_dealer_name'])
def plot_channel_orders(Sales_channel):
  ''' Function for plotting channel orders '''
  df = df1[df1['product_family']=='Smartphone'].copy()
  df_g1 = df.groupby(by=['weeknum_order', Sales_channel]).agg(total = ('order_no','count')).reset_index()
  df_g2 = df.value_counts(subset=[Sales_channel], normalize=False, dropna=False).reset_index()
  top_5 = df_g2.head()[Sales_channel].unique().tolist()
  plt.title(f'50K Hunt. Top 5: {Sales_channel}')
  sns.scatterplot(data=df_g1[df_g1[Sales_channel].isin(top_5)], x='weeknum_order', y='total',
                  hue=Sales_channel) #style=['*','o','v','s','.']
  plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  #plt.xticks(rotation=30)
  formatter = matplotlib.ticker.StrMethodFormatter("{x:.0f}")
  plt.gca().xaxis.set_major_formatter(formatter)
  plt.show()

In [ ]:
# Exploring sales channels
@interact(Sales_channel = ['sales_channel_name','sales_channel_chain_name','sales_channel_dealer_name'])
def split_channel_orders(Sales_channel):
  ''' Function for plotting channel orders '''
  df = df1[df1['product_family']=='Smartphone'].copy()
  df_g1 = df.groupby(by=['weeknum_order', 'order_type', Sales_channel]).agg(total = ('order_no','count')).reset_index()
  df_g2 = df.value_counts(subset=[Sales_channel], normalize=False, dropna=False).reset_index()
  top_5 = df_g2.head(4)[Sales_channel].unique().tolist()
  #plt.title(f'50K Hunt. Top 5: {Sales_channel}')
  sns.catplot(data=df_g1[df_g1[Sales_channel].isin(top_5)], x='weeknum_order', y='total',
              col=Sales_channel, col_wrap=2, ci=False, kind='bar', height=2.5, aspect=1.6) #style=['*','o','v','s','.']
  #plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  #plt.xticks(rotation=30)
  #formatter = matplotlib.ticker.StrMethodFormatter("{x:.0f}")
  #plt.gca().xaxis.set_major_formatter(formatter)
  plt.show()